# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-647ba54792-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Sport Workflow Series ("Association Football Players explorative search") 

Consider the following exploratory information need:

> investigate the association football players and find the main BGPs related to this sport. Compare players per countries, awards of the players and participation in the FIFA World Cup.

**Important consideration: in wikidata, we are talking of ASSOCIATION FOOTBALL**

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q11571`| Cristiano Ronaldo       | node |





Also consider

```
wd:Q11571 ?p ?obj .
```

is the BGP to retrieve all **properties of Cristiano Ronaldo**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should:


1. Identify the BGP for association football player

2. Identify the BGP for association football club

3. Identify the BGP for association football league

4. Return for each european country the number of association football leagues (the result set must be a list of triples country IRI, label and #leagues).

5. Identify the BGP for the ballon d'Or award

6. Return the countries which have at least two different players of their nationality (consider the country of citizenship) who won the ballon d'Or (the result set must be a list of triples country IRI, label and #different players).

7. Identify the BGP of the FIFA World Cup

8. Who are the players who partecipated in more than 4 world's cup editions? (the result set must be a list of triples player IRI, label and #participations).

9. Consider the countries that won the FIFA World Cup at least twice. Find the players of those countries won the ballon d'Or (the result set must be a list of couples player IRI and label).


## Task 1
Identify the BGP for association football player

In [3]:
# I search for the propreties that has Cristiano Ronaldo
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q11571 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P1038'), ('name', 'relative')]
[('p', 'http://www.wikidata.org/prop/direct/P1050'), ('name', 'medical condition')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P109'), ('name', 'signature')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1266'), ('name', 'AlloCiné person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('p', 'http:

In [4]:
#To find the BPG for association football player, I take a look to the occupation of Cristiano ronaldo. 
queryString = """
SELECT DISTINCT ?Occupation ?OccupationName
WHERE {
   # bind something
    wd:Q11571 ?p ?obj.
    ?obj wdt:P106 ?Occupation.
   # get the label
   ?Occupation sc:name ?OccupationName.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('Occupation', 'http://www.wikidata.org/entity/Q22132694'), ('OccupationName', 'museum director')]
[('Occupation', 'http://www.wikidata.org/entity/Q177220'), ('OccupationName', 'singer')]
[('Occupation', 'http://www.wikidata.org/entity/Q43845'), ('OccupationName', 'businessperson')]
[('Occupation', 'http://www.wikidata.org/entity/Q4610556'), ('OccupationName', 'model')]
[('Occupation', 'http://www.wikidata.org/entity/Q937857'), ('OccupationName', 'association football player')]
5


Final query for this task

In [5]:
#The final query for this task it's the same as the last query.
queryString = """
SELECT DISTINCT ?Occupation ?OccupationName
WHERE {
   
    wd:Q11571 ?p ?obj.
    ?obj wdt:P106 ?Occupation.
   # get the label
   ?Occupation sc:name ?OccupationName.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('Occupation', 'http://www.wikidata.org/entity/Q22132694'), ('OccupationName', 'museum director')]
[('Occupation', 'http://www.wikidata.org/entity/Q177220'), ('OccupationName', 'singer')]
[('Occupation', 'http://www.wikidata.org/entity/Q43845'), ('OccupationName', 'businessperson')]
[('Occupation', 'http://www.wikidata.org/entity/Q4610556'), ('OccupationName', 'model')]
[('Occupation', 'http://www.wikidata.org/entity/Q937857'), ('OccupationName', 'association football player')]
5


## Task 2
Identify the BGP for association football club

In [6]:
#I start looking for the properties that association football player has. 
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q937857 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1149'), ('name', 'Library of Congress Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P2163'), ('name', 'FAST ID')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2347'), ('name', 'YSO ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct

In [77]:
#I start looking on the properties of association football player and I look also for the things that are in each property. 
queryString = """
SELECT DISTINCT ?p ?name ?obj ?objectname
WHERE {
   
   wd:Q937857 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?objectname.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P2369'), ('objectname', 'Soccerway player ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P3908'), ('objectname', 'Reprezentacija player ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q18536342'), ('objectname', 'competitive player')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P2446'), ('objectname', 'Transfermarkt player ID')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category"), ('obj', 'http://www.wikidata.org/entity/Q7217464'), ('objectname', 'Category:Association football players')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this

In [79]:
#On the last query I found that on the property part of its association football club, I'm going to look only for the values
#on part of. 
queryString = """
SELECT ?inst ?iname 
WHERE {
   # bind something
   wd:Q937857 ?p ?obj .
   ?obj wdt:P361 ?inst.
   
   # get the label
   ?inst sc:name ?iname.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q841654'), ('iname', 'competition')]
1


I can't find it going that way, I am going to try something else 

In [78]:
#I am going to search the clubs in wich Cristiano Ronaldo played so look at his properties again. 
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q11571 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 200


"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P1038'), ('name', 'relative')]
[('p', 'http://www.wikidata.org/prop/direct/P1050'), ('name', 'medical condition')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P109'), ('name', 'signature')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1266'), ('name', 'AlloCiné person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('p', 'http:

In [80]:
#Now that I have the property "member of sports team"= P54, I search for the values in it. 
queryString = """
SELECT DISTINCT ?teams ?TeamsNames
WHERE {
   # bind something
    wd:Q11571 ?p ?obj.
    ?obj wdt:P54 ?teams .
   # get the label
   ?teams sc:name ?TeamsNames.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('teams', 'http://www.wikidata.org/entity/Q10467'), ('TeamsNames', 'FC Barcelona B')]
[('teams', 'http://www.wikidata.org/entity/Q1543'), ('TeamsNames', 'A.C. Milan')]
[('teams', 'http://www.wikidata.org/entity/Q631'), ('TeamsNames', 'FC Internazionale Milano')]
[('teams', 'http://www.wikidata.org/entity/Q7156'), ('TeamsNames', 'FC Barcelona')]
[('teams', 'http://www.wikidata.org/entity/Q1187790'), ('TeamsNames', 'Argentina national under-20 football team')]
[('teams', 'http://www.wikidata.org/entity/Q11938'), ('TeamsNames', 'PSV Eindhoven')]
[('teams', 'http://www.wikidata.org/entity/Q221882'), ('TeamsNames', "Newell's Old Boys")]
[('teams', 'http://www.wikidata.org/entity/Q2346842'), ('TeamsNames', 'FC Barcelona C')]
[('teams', 'http://www.wikidata.org/entity/Q2462207'), ('TeamsNames', 'Argentina national under-23 football team')]
[('teams', 'http://www.wikidata.org/entity/Q483020'), ('TeamsNames', 'Paris Saint-Germain F.C.')]
[('teams', 'http://www.wikidata.org/entity/Q7980

In [17]:
#Now I am going to search from who is instance of Real Madrid CF "Q8682"
queryString = """
SELECT DISTINCT ?obj ?oName
WHERE {
   # bind something
   wd:Q8682 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?oName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)



Results
[('obj', 'http://www.wikidata.org/entity/Q476028'), ('oName', 'association football club')]
1


Final query for this task

In [18]:
#On the last query I found the solution by searching from what it was instance of Real madrid FC 
queryString = """
SELECT DISTINCT ?obj ?oName
WHERE {
   # bind something
   wd:Q8682 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?oName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q476028'), ('oName', 'association football club')]
1


## Task 3
Identify the BGP for association football league

In [81]:
#On the previous task we explore the properties of Cristiano Ronaldo 
# On the properties there is one called league, so I'm going to start finding what values are on league
queryString = """
SELECT DISTINCT ?obj ?oName
WHERE {
   # bind something
   wd:Q11571 wdt:P118 ?obj .
   # get the label
   ?obj sc:name ?oName.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q182994'), ('oName', 'Liga Portugal')]
[('obj', 'http://www.wikidata.org/entity/Q15804'), ('oName', 'Serie A')]
[('obj', 'http://www.wikidata.org/entity/Q324867'), ('oName', 'La Liga')]
[('obj', 'http://www.wikidata.org/entity/Q9448'), ('oName', 'Premier League')]
4


Final query for this task

In [23]:
#I'm going to see from who it's instance of "La Liga" Q324867
queryString = """
SELECT DISTINCT ?obj ?oName
WHERE {
   # bind something
   wd:Q324867 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?oName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q15991290'), ('oName', 'professional sports league')]
[('obj', 'http://www.wikidata.org/entity/Q15991303'), ('oName', 'association football league')]
2


I found the solution: http://www.wikidata.org/entity/Q15991303 = association football league

## Task 4
Return for each european country the number of association football leagues (the result set must be a list of triples country IRI, label and #leagues).

In [27]:
#I am going to start searching for portugal to find throw portugal, Europe. 
#I take advantage of previous work and I know that cristiano Ronaldo has a property called country of citizenship (P27)
#We take a look at the value that holds that property 
queryString = """
SELECT DISTINCT ?obj ?oName
WHERE {
   # bind something
   wd:Q11571 wdt:P27 ?obj .
   # get the label
   ?obj sc:name ?oName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('obj', 'http://www.wikidata.org/entity/Q45'), ('oName', 'Portugal')]
1


In [9]:
#Now we now the identifier for Portugal, then we look at its properties. 
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q45 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
ORDER BY DESC (?name)
LIMIT 200

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1198'), ('name', 'unemployment rate')]
[('p', 'http://www.wikidata.org/prop/direct/P4841'), ('name', 'total fertility rate')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P78'), ('name', 'top-level Internet domain')]
[('p', 'http://www.wikidata.org/prop/direct/P3984'), ('name', 'subreddit')]
[('p', 'http://www.wikidata.org/prop/direct/P2579'), ('name', 'studied by')]
[('p', 'http://www.wikidata.org/prop/direct/P47'), ('name', 'shares border with')]
[('p', 'http://www.wikidata.org/prop/direct/P3001'), ('name', 'retirement age')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P5658'), ('name', 'railway traffic side')]
[('p', 'http://www.wikidata.org/prop/direct/P832'), ('name', 'public 

In [31]:
#We are going to find from who its instance of 
queryString = """
SELECT DISTINCT ?obj ?oName
WHERE {
   # bind something
   wd:Q45 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?oName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q20181813'), ('oName', 'colonial power')]
[('obj', 'http://www.wikidata.org/entity/Q3624078'), ('oName', 'sovereign state')]
[('obj', 'http://www.wikidata.org/entity/Q6256'), ('oName', 'country')]
3


In [33]:
#We find country but not Europe so we take a look on the rest of properties and we find one called continen P30 so we take 
#a look at its value
queryString = """
SELECT DISTINCT ?obj ?oName
WHERE {
   # bind something
   wd:Q45 wdt:P30 ?obj .
   # get the label
   ?obj sc:name ?oName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)



Results
[('obj', 'http://www.wikidata.org/entity/Q46'), ('oName', 'Europe')]
1


In [73]:
#We get all instance from association football leagues, that is going to give me all the leagues of the world. 
queryString = """

SELECT DISTINCT ?footballleague ?footballleagueName 
WHERE {

   #bind something 
    ?footballleague wdt:P31 wd:Q15991303 . #All the leagues in the world  
    
   # get the label 
   ?footballleague sc:name ?footballleagueName . 
   
}

LIMIT 3


"""

print("Results")
x=run_query(queryString)

Results
[('footballleague', 'http://www.wikidata.org/entity/Q304228'), ('footballleagueName', 'Uzbekistan Pro League')]
[('footballleague', 'http://www.wikidata.org/entity/Q2950655'), ('footballleagueName', 'Ligue 2')]
[('footballleague', 'http://www.wikidata.org/entity/Q2951924'), ('footballleagueName', "Moldovan women's football championship")]
3


In [75]:
#Now we explore the properties that a league has searching for the propertie country. 
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q304228 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P2257'), ('name', 'event interval')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P571'), ('name', 'inception')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('name', 'Freebase ID')]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('name', 'organizer')]
[('p', 'http://www.wikidata.org/prop/direct/P856'), ('name', 'official website')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
[('p', 'http://www.wikidata.org/prop/direct/P2499'), ('name', 'league level above')]
11


In [91]:
#Now we add to the previus query the country of each league. 
queryString = """

SELECT DISTINCT ?footballleague ?footballleagueName ?countryleagueName
WHERE {

   #bind something 
    ?footballleague wdt:P31 wd:Q15991303 . #All the leagues in the world  
    ?footballleague wdt:P17 ?countryleague . #Put the country of the league on the variable countryleague
    
   # get the label 
   ?footballleague sc:name ?footballleagueName . 
   ?countryleague sc:name ?countryleagueName.
}
ORDER BY ASC (?countryleagueName)
 
LIMIT 10


"""

print("Results")
x=run_query(queryString)

Results
[('footballleague', 'http://www.wikidata.org/entity/Q170215'), ('footballleagueName', 'Afghan Premier League'), ('countryleagueName', 'Afghanistan')]
[('footballleague', 'http://www.wikidata.org/entity/Q1383952'), ('footballleagueName', 'Albanian Second Division'), ('countryleagueName', 'Albania')]
[('footballleague', 'http://www.wikidata.org/entity/Q116909'), ('footballleagueName', 'Albanian First Division'), ('countryleagueName', 'Albania')]
[('footballleague', 'http://www.wikidata.org/entity/Q1349507'), ('footballleagueName', 'Albanian Third Division'), ('countryleagueName', 'Albania')]
[('footballleague', 'http://www.wikidata.org/entity/Q647746'), ('footballleagueName', 'Algerian Ligue Professionnelle 1'), ('countryleagueName', 'Algeria')]
[('footballleague', 'http://www.wikidata.org/entity/Q2573693'), ('footballleagueName', 'Algerian Ligue Professionnelle 2'), ('countryleagueName', 'Algeria')]
[('footballleague', 'http://www.wikidata.org/entity/Q3240645'), ('footballleague

In [94]:
#Now we only show the countries and how many leagues each country have.  
queryString = """

SELECT ?countryleagueName (COUNT(*) AS ?total)
WHERE {

   #bind something 
    ?footballleague wdt:P31 wd:Q15991303 . #All the leagues in the world  
    ?footballleague wdt:P17 ?countryleague . #Put the country of the league on the variable countryleague
   
   # get the label 
   
   ?countryleague sc:name ?countryleagueName.
}
GROUP BY ?countryleagueName
ORDER BY ASC (?countryleagueName)
LIMIT 10


"""

print("Results")
x=run_query(queryString)


Results
[('countryleagueName', 'Afghanistan'), ('total', '1')]
[('countryleagueName', 'Albania'), ('total', '3')]
[('countryleagueName', 'Algeria'), ('total', '7')]
[('countryleagueName', 'Andorra'), ('total', '2')]
[('countryleagueName', 'Angola'), ('total', '1')]
[('countryleagueName', 'Antigua and Barbuda'), ('total', '1')]
[('countryleagueName', 'Argentina'), ('total', '21')]
[('countryleagueName', 'Armenia'), ('total', '2')]
[('countryleagueName', 'Aruba'), ('total', '1')]
[('countryleagueName', 'Australia'), ('total', '19')]
10


Final query for this task

In [101]:
#Now we filter the coutries to only the european ones. 
queryString = """

SELECT ?country ?countryName (COUNT(*) AS ?numberofLeagues)
WHERE {

   #bind something 
    ?footballleague wdt:P31 wd:Q15991303 . #All the leagues in the world  
    ?footballleague wdt:P17 ?country . #Put the country of the league on the variable countryleague
    ?country wdt:P30 wd:Q46 . #On the property continent(P30) the value has to be europe(Q46)
   # get the label 
   
   ?country sc:name ?countryName.
}

ORDER BY ASC (?countryName) 
LIMIT 100


"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q222'), ('countryName', 'Albania'), ('numberofLeagues', '3')]
[('country', 'http://www.wikidata.org/entity/Q228'), ('countryName', 'Andorra'), ('numberofLeagues', '2')]
[('country', 'http://www.wikidata.org/entity/Q399'), ('countryName', 'Armenia'), ('numberofLeagues', '2')]
[('country', 'http://www.wikidata.org/entity/Q40'), ('countryName', 'Austria'), ('numberofLeagues', '12')]
[('country', 'http://www.wikidata.org/entity/Q227'), ('countryName', 'Azerbaijan'), ('numberofLeagues', '2')]
[('country', 'http://www.wikidata.org/entity/Q184'), ('countryName', 'Belarus'), ('numberofLeagues', '3')]
[('country', 'http://www.wikidata.org/entity/Q31'), ('countryName', 'Belgium'), ('numberofLeagues', '11')]
[('country', 'http://www.wikidata.org/entity/Q225'), ('countryName', 'Bosnia and Herzegovina'), ('numberofLeagues', '8')]
[('country', 'http://www.wikidata.org/entity/Q219'), ('countryName', 'Bulgaria'), ('numberofLeagues', '4')]
[('country

## Task 5
Identify the BGP for the ballon d'Or award

Final query for this task

In [35]:
#Explorin the previous query that provides all the properties of Cristiano Ronaldo, I find the property P166 "award received".
#Let's see what values contains. 
queryString = """
SELECT DISTINCT ?obj ?oName
WHERE {
   # bind something
   wd:Q11571 wdt:P166 ?obj .
   # get the label
   ?obj sc:name ?oName.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1750244'), ('oName', 'FWA Footballer of the Year')]
[('obj', 'http://www.wikidata.org/entity/Q10726649'), ('oName', "PFA Fans' Player of the Year")]
[('obj', 'http://www.wikidata.org/entity/Q20888805'), ('oName', 'Commander of the Order of Merit of Portugal')]
[('obj', 'http://www.wikidata.org/entity/Q1146578'), ('oName', 'UEFA Team of the Year')]
[('obj', 'http://www.wikidata.org/entity/Q182529'), ('oName', 'FIFA World Player of the Year')]
[('obj', 'http://www.wikidata.org/entity/Q260117'), ('oName', "UEFA Men's Player of the Year Award")]
[('obj', 'http://www.wikidata.org/entity/Q166177'), ('oName', "Ballon d'Or")]
[('obj', 'http://www.wikidata.org/entity/Q20860117'), ('oName', 'Officer of the Order of Prince Henry')]
[('obj', 'http://www.wikidata.org/entity/Q20859673'), ('oName', 'Grand Officer of the Order of Prince Henry')]
[('obj', 'http://www.wikidata.org/entity/Q2986505'), ('oName', 'Sir Matt Busby Player of the Year')]
[('obj'

We find the ballon d'or among Cristiano Ronaldo's awards.  

## Task 6
Return the countries which have at least two different players of their nationality (consider the country of citizenship) who won the ballon d'Or (the result set must be a list of triples country IRI, label and #different players).

In [62]:
#I'm going to start finding all the persons that are football players
#I take advantage of the previous queries and use the properties P106(Ocupation) and the value Q937857 (association football player)

queryString = """

SELECT DISTINCT ?footballplayers ?footballplayersName 
WHERE {

   #bind something 
   ?footballplayers wdt:P31 wd:Q5 . #I filter all the humans in the variable footballplayers
   ?footballplayers wdt:P106 wd:Q937857 . #now I add the condition to the occupation has to be "association football player"
   
   # get the label 
   ?footballplayers sc:name ?footballplayersName . 
   
}
#I put the limit in 20 because there are going to be a lot of football players. 
LIMIT 20 

"""

print("Results")
x=run_query(queryString)

Results
[('footballplayers', 'http://www.wikidata.org/entity/Q259758'), ('footballplayersName', 'Andrés Mendoza')]
[('footballplayers', 'http://www.wikidata.org/entity/Q8135287'), ('footballplayersName', 'Naoki Kurimoto')]
[('footballplayers', 'http://www.wikidata.org/entity/Q8157342'), ('footballplayersName', 'Lamberto Anichini')]
[('footballplayers', 'http://www.wikidata.org/entity/Q8196829'), ('footballplayersName', 'Alvin Avinesh')]
[('footballplayers', 'http://www.wikidata.org/entity/Q8199060'), ('footballplayersName', 'André Advíncula')]
[('footballplayers', 'http://www.wikidata.org/entity/Q8199367'), ('footballplayersName', 'Andrés Mendoza')]
[('footballplayers', 'http://www.wikidata.org/entity/Q8201109'), ('footballplayersName', 'Antonino Gart')]
[('footballplayers', 'http://www.wikidata.org/entity/Q8204020'), ('footballplayersName', 'Argemiro Veiga Gonçalves')]
[('footballplayers', 'http://www.wikidata.org/entity/Q8238626'), ('footballplayersName', 'Jiří Adamec')]
[('footballp

In [61]:
#I use the previous query and add those football players that won the ballon d'ore. 
#I take the value of ballon d'ore from the task 5. Q166177 
queryString = """

SELECT DISTINCT ?footballplayers ?footballplayersName  
WHERE {
   # bind something
   ?footballplayers wdt:P31 wd:Q5 . #I filter all the humans in the variable footballplayers
   ?footballplayers wdt:P106 wd:Q937857 . #now I add the condition to the occupation has to be "association football player"
   ?footballplayers wdt:P166 wd:Q166177 . #with ballon d'or in the property P166 (award recevied)
   
   # get the label 
   ?footballplayers sc:name ?footballplayersName . 
   
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('footballplayers', 'http://www.wikidata.org/entity/Q162043'), ('footballplayersName', 'Eusébio')]
[('footballplayers', 'http://www.wikidata.org/entity/Q128829'), ('footballplayersName', 'Michael Owen')]
[('footballplayers', 'http://www.wikidata.org/entity/Q102027'), ('footballplayersName', 'Fabio Cannavaro')]
[('footballplayers', 'http://www.wikidata.org/entity/Q36386'), ('footballplayersName', 'Lothar Matthäus')]
[('footballplayers', 'http://www.wikidata.org/entity/Q178628'), ('footballplayersName', 'Paolo Rossi')]
[('footballplayers', 'http://www.wikidata.org/entity/Q216140'), ('footballplayersName', 'Stanley Matthews')]
[('footballplayers', 'http://www.wikidata.org/entity/Q310657'), ('footballplayersName', 'Matías Fernandez')]
[('footballplayers', 'http://www.wikidata.org/entity/Q483145'), ('footballplayersName', 'Luís Figo')]
[('footballplayers', 'http://www.wikidata.org/entity/Q483577'), ('footballplayersName', 'Rivaldo')]
[('footballplayers', 'http://www.wikidata.org/en

In [60]:
#Now I add the country of each player. 
#We take the property of country from the properties that has Cristiano Ronaldo.  P27
queryString = """

SELECT DISTINCT ?futballplayers ?footballplayersName ?countryName 
WHERE {
   # bind something
   ?footballplayers wdt:P31 wd:Q5 . #I filter all the humans in the variable footballplayers
   ?footballplayers wdt:P106 wd:Q937857 . #now I add the condition to the occupation has to be "association football player"
   ?footballplayers wdt:P166 wd:Q166177 . #with ballon d'or in the property P166 (award recevied)
   ?footballplayers wdt:P27 ?country . #Now I take the countries of the players and put it on the variable country.
   
   # get the label 
   ?footballplayers sc:name ?footballplayersName . 
   ?country sc:name ?countryName . 
}
ORDER BY ASC (?countryName)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('footballplayersName', 'Zinedine Zidane'), ('countryName', 'Algeria')]
[('footballplayersName', 'Omar Sívori'), ('countryName', 'Argentina')]
[('footballplayersName', 'Lionel Messi'), ('countryName', 'Argentina')]
[('footballplayersName', 'Alfredo Di Stéfano'), ('countryName', 'Argentina')]
[('footballplayersName', 'Rivaldo'), ('countryName', 'Brazil')]
[('footballplayersName', 'Ronaldinho'), ('countryName', 'Brazil')]
[('footballplayersName', 'Kaká'), ('countryName', 'Brazil')]
[('footballplayersName', 'Ronaldo'), ('countryName', 'Brazil')]
[('footballplayersName', 'Hristo Stoichkov'), ('countryName', 'Bulgaria')]
[('footballplayersName', 'Matías Fernandez'), ('countryName', 'Chile')]
[('footballplayersName', 'Alfredo Di Stéfano'), ('countryName', 'Colombia')]
[('footballplayersName', 'Luka Modrić'), ('countryName', 'Croatia')]
[('footballplayersName', 'Pavel Nedvěd'), ('countryName', 'Czech Republic')]
[('footballplayersName', 'Josef Masopust'), ('countryName', 'Czech Repub

In [59]:
#Now I only show the countries 
queryString = """

SELECT DISTINCT ?country ?countryName 
WHERE {
   # bind something
   ?footballplayers wdt:P31 wd:Q5 . #every human
   ?footballplayers wdt:P106 wd:Q937857 . #with occupation association football player
   ?footballplayers wdt:P166 wd:Q166177 . #with ballon d'or in the property P166 (award recevied)
   ?footballplayers wdt:P27 ?country . #Now we take the countries of the players and put it on the variable country.
   
   # get the label 
   ?country sc:name ?countryName . 
}

ORDER BY ASC (?countryName)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q262'), ('countryName', 'Algeria')]
[('country', 'http://www.wikidata.org/entity/Q414'), ('countryName', 'Argentina')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('countryName', 'Brazil')]
[('country', 'http://www.wikidata.org/entity/Q219'), ('countryName', 'Bulgaria')]
[('country', 'http://www.wikidata.org/entity/Q298'), ('countryName', 'Chile')]
[('country', 'http://www.wikidata.org/entity/Q739'), ('countryName', 'Colombia')]
[('country', 'http://www.wikidata.org/entity/Q224'), ('countryName', 'Croatia')]
[('country', 'http://www.wikidata.org/entity/Q213'), ('countryName', 'Czech Republic')]
[('country', 'http://www.wikidata.org/entity/Q33946'), ('countryName', 'Czechoslovakia')]
[('country', 'http://www.wikidata.org/entity/Q35'), ('countryName', 'Denmark')]
[('country', 'http://www.wikidata.org/entity/Q21'), ('countryName', 'England')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France')]
[('cou

In [58]:
#Now I try to count the winners for each country and show the number. 
queryString = """

SELECT DISTINCT ?country ?countryName (COUNT(*) AS ?winners)
WHERE {
   # bind something
   ?footballplayers wdt:P31 wd:Q5 . #every human
   ?footballplayers wdt:P106 wd:Q937857 . #with occupation association football player
   ?footballplayers wdt:P166 wd:Q166177 . #with ballon d'or in the property P166 (award recevied)
   ?footballplayers wdt:P27 ?country . #Now we take the countries of the players and put it on the variable country.
   
   # get the label 
   ?country sc:name ?countryName . 
}
ORDER BY ASC (?countryName)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q262'), ('countryName', 'Algeria'), ('winners', '1')]
[('country', 'http://www.wikidata.org/entity/Q414'), ('countryName', 'Argentina'), ('winners', '3')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('countryName', 'Brazil'), ('winners', '4')]
[('country', 'http://www.wikidata.org/entity/Q219'), ('countryName', 'Bulgaria'), ('winners', '1')]
[('country', 'http://www.wikidata.org/entity/Q298'), ('countryName', 'Chile'), ('winners', '1')]
[('country', 'http://www.wikidata.org/entity/Q739'), ('countryName', 'Colombia'), ('winners', '1')]
[('country', 'http://www.wikidata.org/entity/Q224'), ('countryName', 'Croatia'), ('winners', '1')]
[('country', 'http://www.wikidata.org/entity/Q213'), ('countryName', 'Czech Republic'), ('winners', '2')]
[('country', 'http://www.wikidata.org/entity/Q33946'), ('countryName', 'Czechoslovakia'), ('winners', '1')]
[('country', 'http://www.wikidata.org/entity/Q35'), ('countryName', 'Denmark'), ('wi

In [57]:
#Now I filter to only show the countries with more than 2 players that won the ballon d'or 
queryString = """

SELECT DISTINCT ?country ?countryName (COUNT(*) AS ?winners)
WHERE {
   # bind something
   ?footballplayers wdt:P31 wd:Q5 . #every human
   ?footballplayers wdt:P106 wd:Q937857 . #with occupation association football player
   ?footballplayers wdt:P166 wd:Q166177 . #with ballon d'or in the property P166 (award recevied)
   ?footballplayers wdt:P27 ?country . #Now we take the countries of the players and put it on the variable country.
   
   # get the label 
   ?country sc:name ?countryName . 
   FILTER (?winners >2)  
}
ORDER BY ASC (?countryName)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
Empty


In [ ]:
I realiced that the query gave me the empty result because I can`t filter like that, I look for information and it`s with having. 

Final query for this task

In [56]:
# write your final query
#Now I filter to only show the countries with more than 2 players that won the ballon d'or 
queryString = """

SELECT DISTINCT ?country ?countryName (COUNT(*) AS ?winners)
WHERE {
   # bind something
   ?footballplayers wdt:P31 wd:Q5 . #every human
   ?footballplayers wdt:P106 wd:Q937857 . #with occupation association football player
   ?footballplayers wdt:P166 wd:Q166177 . #with ballon d'or in the property P166 (award recevied)
   ?footballplayers wdt:P27 ?country . #Now we take the countries of the players and put it on the variable country.
   
   # get the label 
   ?country sc:name ?countryName . 
     
}
GROUP BY ?country ?countryName
HAVING (count(*) >= 2)
ORDER BY ASC (?countryName)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q414'), ('countryName', 'Argentina'), ('winners', '3')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('countryName', 'Brazil'), ('winners', '4')]
[('country', 'http://www.wikidata.org/entity/Q213'), ('countryName', 'Czech Republic'), ('winners', '2')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('winners', '4')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany'), ('winners', '5')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy'), ('winners', '6')]
[('country', 'http://www.wikidata.org/entity/Q29999'), ('countryName', 'Kingdom of the Netherlands'), ('winners', '3')]
[('country', 'http://www.wikidata.org/entity/Q45'), ('countryName', 'Portugal'), ('winners', '3')]
[('country', 'http://www.wikidata.org/entity/Q15180'), ('countryName', 'Soviet Union'), ('winners', '3')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('countryName', '

## Task 7
Identify the BGP of the FIFA World Cup

In [36]:
#Again, I search among the properties of Cristiano Ronaldo and I find a property called, Participan in (P1344)
#Let's find what contains. 
queryString = """
SELECT DISTINCT ?obj ?oName
WHERE {
   # bind something
   wd:Q11571 wdt:P1344 ?obj .
   # get the label
   ?obj sc:name ?oName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q373501'), ('oName', 'UEFA Euro 2020')]
[('obj', 'http://www.wikidata.org/entity/Q102920'), ('oName', 'UEFA Euro 2004')]
[('obj', 'http://www.wikidata.org/entity/Q170645'), ('oName', '2018 FIFA World Cup')]
[('obj', 'http://www.wikidata.org/entity/Q176883'), ('oName', '2010 FIFA World Cup')]
[('obj', 'http://www.wikidata.org/entity/Q189571'), ('oName', 'UEFA Euro 2016')]
[('obj', 'http://www.wikidata.org/entity/Q22669'), ('oName', 'UEFA Euro 2012')]
[('obj', 'http://www.wikidata.org/entity/Q241864'), ('oName', 'UEFA Euro 2008')]
[('obj', 'http://www.wikidata.org/entity/Q324867'), ('oName', 'La Liga')]
[('obj', 'http://www.wikidata.org/entity/Q37285'), ('oName', '2006 FIFA World Cup')]
[('obj', 'http://www.wikidata.org/entity/Q79859'), ('oName', '2014 FIFA World Cup')]
[('obj', 'http://www.wikidata.org/entity/Q8558'), ('oName', '2004 Summer Olympics')]
11


In [37]:
#Lets explore one of the FIFA WORLD CUP and see their properties. In this case the 2006 Fifa world cup (Q37285)
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q37285 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1350'), ('name', 'number of matches played/races/starts')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1546'), ('name', 'motto')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('name', 'competition class')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('name', 'VIAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[

In [38]:
#Lets see from whom it's instance of
queryString = """
SELECT DISTINCT ?obj ?oName
WHERE {
   # bind something
   wd:Q37285 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?oName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q27020041'), ('oName', 'sports season')]
1


Final query for this task

In [39]:
#I explore the property sports season of league or competition (P3450)
queryString = """
SELECT DISTINCT ?obj ?oName
WHERE {
   # bind something
   wd:Q37285 wdt:P3450 ?obj .
   # get the label
   ?obj sc:name ?oName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q19317'), ('oName', 'FIFA World Cup')]
1


## Task 8
Who are the players who partecipated in more than 4 world's cup editions? (the result set must be a list of triples player IRI, label and #participations).

On task 7, while exploring finding the BGP of the FIFA WORLD CUP, I realice that there isn't only the FIFA WORLD CUP, istead for each year there is a 
Fifa world cup, for example, I find on the first query: 

        - 2006 Fifa world cup (Q37285)
        - 2010 Fifa world cup (Q176883)
        - 2014 Fifa world cup (Q79859)
        - 2018 Fifa world cup (Q170645)
I'm going to take advantage of this task and use some of the properties that I think are going to be the most values ones.

    - sports season of league or competition (P3450)
    - FIFA WORLD CUP (Q19317)
From previous ones, The tasks: 

    - Participan in (P1344), property on all players.
    
    

In [3]:
#I get the same query that in the task 6 to get all the players. 
queryString = """

SELECT DISTINCT ?footballplayers ?footballplayersName 
WHERE {

   #bind something
   ?footballplayers wdt:P31 wd:Q5 . #I filter all the humans in the variable futballplayers
   ?footballplayers wdt:P106 wd:Q937857 . #now I add the condition to the occupation has to be "association football player"
   
   # get the label 
   ?footballplayers sc:name ?footballplayersName . 
   
}

#I put the limit in 20 because there are going to be a lot of football players. 
LIMIT 3

"""

print("Results")
x=run_query(queryString)

Results
[('futballplayers', 'http://www.wikidata.org/entity/Q259758'), ('futballplayersName', 'Andrés Mendoza')]
[('futballplayers', 'http://www.wikidata.org/entity/Q8135287'), ('futballplayersName', 'Naoki Kurimoto')]
[('futballplayers', 'http://www.wikidata.org/entity/Q8157342'), ('futballplayersName', 'Lamberto Anichini')]
3


In [55]:
#Now I am gonna explore all the tournaments that the players participated. 
queryString = """

SELECT DISTINCT ?footballplayers ?footballplayersName ?participatedName
WHERE {

   # bind something
   ?footballplayers wdt:P31 wd:Q5 . #I filter all the humans in the variable futballplayers
   ?footballplayers wdt:P106 wd:Q937857 . #now I add the condition to the occupation has to be "association football player"
   ?footballplayers wdt:P1344 ?participated . #Now I add all the values that a football player has in the property to the variable ?participated
   
   
   # get the label 
   ?footballplayers sc:name ?footballplayersName . 
   ?participated sc:name ?participatedName . 
   
}
ORDER BY ?footballplayersName
#I put the limit in 20 because there are going to be a lot of football players. 
LIMIT 3


"""

print("Results")
x=run_query(queryString)

Results
[('footballplayers', 'http://www.wikidata.org/entity/Q1994404'), ('footballplayersName', 'A. Haslam'), ('participatedName', '1900 Summer Olympics')]
[('footballplayers', 'http://www.wikidata.org/entity/Q4647992'), ('footballplayersName', 'A. J. Wood'), ('participatedName', '1996 Summer Olympics')]
[('footballplayers', 'http://www.wikidata.org/entity/Q12300163'), ('footballplayersName', 'Aage Andersen'), ('participatedName', '1906 Intercalated Games')]
3


In [31]:
#Now I am going to filter the variable ?participated to get only the players that participated on a Fifa world cup 
queryString = """

SELECT DISTINCT ?footballplayers ?footballplayersName ?participatedName
WHERE {

   # bind something
   ?footballplayers wdt:P31 wd:Q5 . #I filter all the humans in the variable futballplayers
   ?footballplayers wdt:P106 wd:Q937857 . #now I add the condition to the occupation has to be "association football player"
   ?footballplayers wdt:P1344 ?participated . #Now I add all the values that a football player has in the property to the variable ?participated
   ?participated wdt:P3450 wd:Q19317.
   
   # get the label 
   ?footballplayers sc:name ?footballplayersName . 
   ?participated sc:name ?participatedName . 
   
}
GROUP BY ?footballplayers 
ORDER BY ?footballplayersName
#I put the limit in 20 because there are going to be a lot of football players. 
LIMIT 30


"""

print("Results")
x=run_query(queryString)

Results
[('footballplayers', 'http://www.wikidata.org/entity/Q301865'), ('footballplayersName', 'Aaron Clapham'), ('participatedName', '2010 FIFA World Cup')]
[('footballplayers', 'http://www.wikidata.org/entity/Q431021'), ('footballplayersName', 'Aaron Lawrence'), ('participatedName', '1998 FIFA World Cup')]
[('footballplayers', 'http://www.wikidata.org/entity/Q175303'), ('footballplayersName', 'Aaron Lennon'), ('participatedName', '2010 FIFA World Cup')]
[('footballplayers', 'http://www.wikidata.org/entity/Q175303'), ('footballplayersName', 'Aaron Lennon'), ('participatedName', '2006 FIFA World Cup')]
[('footballplayers', 'http://www.wikidata.org/entity/Q182549'), ('footballplayersName', 'Aaron Mokoena'), ('participatedName', '2010 FIFA World Cup')]
[('footballplayers', 'http://www.wikidata.org/entity/Q182549'), ('footballplayersName', 'Aaron Mokoena'), ('participatedName', '2002 FIFA World Cup')]
[('footballplayers', 'http://www.wikidata.org/entity/Q478941'), ('footballplayersName',

In [53]:
#Now I tried to count the times each player appears.
queryString = """

SELECT DISTINCT ?footballplayers ?footballplayersName (COUNT(*) AS ?Times)
WHERE {

   # bind something
   ?footballplayers wdt:P31 wd:Q5 . #I filter all the humans in the variable futballplayers
   ?footballplayers wdt:P106 wd:Q937857 . #now I add the condition to the occupation has to be "association football player"
   ?footballplayers wdt:P1344 ?participated . #Now I add all the values that a football player has in the property to the variable ?participated
   ?participated wdt:P3450 wd:Q19317.
   
   # get the label 
   ?footballplayers sc:name ?footballplayersName . 
   ?participated sc:name ?participatedName . 
   
}
GROUP BY ?footballplayers ?footballplayersName 
ORDER BY DESC (?Times)
#I put the limit in 20 because there are going to be a lot of football players. 
LIMIT 100


"""

print("Results")
x=run_query(queryString)

Results
[('footballplayers', 'http://www.wikidata.org/entity/Q36386'), ('footballplayersName', 'Lothar Matthäus'), ('Times', '5')]
[('footballplayers', 'http://www.wikidata.org/entity/Q186330'), ('footballplayersName', 'Rafael Márquez'), ('Times', '5')]
[('footballplayers', 'http://www.wikidata.org/entity/Q265428'), ('footballplayersName', 'Antonio Carbajal'), ('Times', '5')]
[('footballplayers', 'http://www.wikidata.org/entity/Q68060'), ('footballplayersName', 'Gianluigi Buffon'), ('Times', '5')]
[('footballplayers', 'http://www.wikidata.org/entity/Q312179'), ('footballplayersName', 'Mohamed Al-Deayea'), ('Times', '4')]
[('footballplayers', 'http://www.wikidata.org/entity/Q516417'), ('footballplayersName', 'Pedro Rocha'), ('Times', '4')]
[('footballplayers', 'http://www.wikidata.org/entity/Q313035'), ('footballplayersName', 'DaMarcus Beasley'), ('Times', '4')]
[('footballplayers', 'http://www.wikidata.org/entity/Q217389'), ('footballplayersName', 'Valon Behrami'), ('Times', '4')]
[('f

Final query for this task

In [54]:
#Now I add a filter to show only the players that have been in more than 4 world cups.
queryString = """

SELECT DISTINCT ?footballplayers ?footballplayersName (COUNT(*) AS ?Times)
WHERE {

   # bind something
   ?footballplayers wdt:P31 wd:Q5 . #I filter all the humans in the variable futballplayers
   ?footballplayers wdt:P106 wd:Q937857 . #now I add the condition to the occupation has to be "association football player"
   ?footballplayers wdt:P1344 ?participated . #Now I add all the values that a football player has in the property to the variable ?participated
   ?participated wdt:P3450 wd:Q19317.
   
   # get the label 
   ?footballplayers sc:name ?footballplayersName . 
   ?participated sc:name ?participatedName . 
   
}
GROUP BY ?footballplayers ?footballplayersName 
HAVING (COUNT(*) >4 ) #With having I can limit the times. 
ORDER BY DESC (?Times)
#I put the limit in 20 because there are going to be a lot of football players. 
LIMIT 30


"""

print("Results")
x=run_query(queryString)


Results
[('footballplayers', 'http://www.wikidata.org/entity/Q36386'), ('footballplayersName', 'Lothar Matthäus'), ('Times', '5')]
[('footballplayers', 'http://www.wikidata.org/entity/Q186330'), ('footballplayersName', 'Rafael Márquez'), ('Times', '5')]
[('footballplayers', 'http://www.wikidata.org/entity/Q265428'), ('footballplayersName', 'Antonio Carbajal'), ('Times', '5')]
[('footballplayers', 'http://www.wikidata.org/entity/Q68060'), ('footballplayersName', 'Gianluigi Buffon'), ('Times', '5')]
4


## Task 9
Consider the countries that won the FIFA World Cup at least twice. Find the players of those countries won the ballon d'Or (the result set must be a list of couples player IRI and label).

Firts I'm going to search for all the editions of the World Cup, I'm going to use some properties and identifiers from previus tasks like:
 
    - sports season of league or competition (P3450)
    - FIFA WORLD CUP (Q19317)
    - 2006 Fifa world cup (Q37285)
    - 2010 Fifa world cup (Q176883)
    - Participan in (P1344), property on all players.
    

In [8]:
#We are going to start to find all the editions of the world Cup. For that we explore the properties from the 2006 FIFA WORLD CUP 
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q37285 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1350'), ('name', 'number of matches played/races/starts')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1546'), ('name', 'motto')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('name', 'competition class')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('name', 'VIAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[

In [13]:
#Now I explore from who it's instance the 2010 FIFA WORLD CUP
queryString = """
SELECT DISTINCT ?inst ?instName
WHERE {
   # bind something
    wd:Q176883 ?p ?obj.
    ?obj wdt:P31 ?inst.
   # get the label
   ?inst sc:name ?instName.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q22936940'), ('instName', 'competition class')]
[('inst', 'http://www.wikidata.org/entity/Q15303456'), ('instName', 'rugby union venue')]
[('inst', 'http://www.wikidata.org/entity/Q11422536'), ('instName', 'international sport governing body')]
[('inst', 'http://www.wikidata.org/entity/Q37178026'), ('instName', 'meta-organization')]
[('inst', 'http://www.wikidata.org/entity/Q1478443'), ('instName', 'association football federation')]
[('inst', 'http://www.wikidata.org/entity/Q11753321'), ('instName', 'Wikimedia navigational template')]
[('inst', 'http://www.wikidata.org/entity/Q163740'), ('instName', 'nonprofit organization')]
[('inst', 'http://www.wikidata.org/entity/Q212434'), ('instName', 'Olympic sport')]
[('inst', 'http://www.wikidata.org/entity/Q31629'), ('instName', 'type of sport')]
[('inst', 'http://www.wikidata.org/entity/Q3624078'), ('instName', 'sovereign state')]
[('inst', 'http://www.wikidata.org/entity/Q4167836'), ('instN

In [22]:
#Now I get all the values instance of World championship. 
queryString = """

SELECT DISTINCT ?allworldcups ?allworldcupsName 
WHERE {

   #bind something
   ?allworldcups wdt:P31 wd:Q1344963 . #I filter all the world championships in the variable world Championship
   #?footballplayers wdt:P106 wd:Q937857 . #now I add the condition to the occupation has to be "association football player"
   
   # get the label 
   ?allworldcups sc:name ?allworldcupsName  . 
   
}
ORDER BY ?allworldcupsName 
#I put the limit in 20 because there are going to be a lot of football players. 
LIMIT 500


"""

print("Results")
x=run_query(queryString)

Results
[('allworldcups', 'http://www.wikidata.org/entity/Q63819736'), ('allworldcupsName', '1912 Balkline 45.2 World Championship')]
[('allworldcups', 'http://www.wikidata.org/entity/Q63819741'), ('allworldcupsName', '1919 Balkline 45.2 World Championship')]
[('allworldcups', 'http://www.wikidata.org/entity/Q84432381'), ('allworldcupsName', '1934 world one-cushion championship #1')]
[('allworldcups', 'http://www.wikidata.org/entity/Q84432929'), ('allworldcupsName', '1934 world one-cushion championship #2')]
[('allworldcups', 'http://www.wikidata.org/entity/Q84473934'), ('allworldcupsName', '1935 1-cushion world championship')]
[('allworldcups', 'http://www.wikidata.org/entity/Q84474102'), ('allworldcupsName', '1937 1-cushion world championship')]
[('allworldcups', 'http://www.wikidata.org/entity/Q84474125'), ('allworldcupsName', '1968 1-cushion world championship')]
[('allworldcups', 'http://www.wikidata.org/entity/Q84474142'), ('allworldcupsName', '1974 1-cushion world championship')

This last query doesn't saw me what I want. I try another path. 

In [20]:
#I remember the property sports season of league or competition(P345O) from previous tasks and I try to show all the world cups 
queryString = """

SELECT DISTINCT ?allworldcups ?allworldcupsName 
WHERE {

   #bind something
   ?allworldcups wdt:P3450 wd:Q19317 . #I filter all the world championships in the variable world Championship
   
   
   # get the label 
   ?allworldcups sc:name ?allworldcupsName  . 
   
}
ORDER BY ?allworldcupsName 
LIMIT 100


"""

print("Results")
x=run_query(queryString)

Results
[('allworldcups', 'http://www.wikidata.org/entity/Q48432'), ('allworldcupsName', '1930 FIFA World Cup')]
[('allworldcups', 'http://www.wikidata.org/entity/Q1477177'), ('allworldcupsName', '1934 FIFA World Cup')]
[('allworldcups', 'http://www.wikidata.org/entity/Q131591'), ('allworldcupsName', '1938 FIFA World Cup')]
[('allworldcups', 'http://www.wikidata.org/entity/Q203851'), ('allworldcupsName', '1942 FIFA World Cup')]
[('allworldcups', 'http://www.wikidata.org/entity/Q65273310'), ('allworldcupsName', '1946 FIFA World Cup')]
[('allworldcups', 'http://www.wikidata.org/entity/Q132515'), ('allworldcupsName', '1950 FIFA World Cup')]
[('allworldcups', 'http://www.wikidata.org/entity/Q131075'), ('allworldcupsName', '1954 FIFA World Cup')]
[('allworldcups', 'http://www.wikidata.org/entity/Q132533'), ('allworldcupsName', '1958 FIFA World Cup')]
[('allworldcups', 'http://www.wikidata.org/entity/Q160813'), ('allworldcupsName', '1962 FIFA World Cup')]
[('allworldcups', 'http://www.wikida

In [21]:
#Now that I have all the World cups, I try to get the country that won each one of them. From last queries I know the property winner(P1346) lets get
#the winners 
queryString = """

SELECT DISTINCT ?allworldcups ?allworldcupsName ?winner ?winnerName 
WHERE {

   #bind something
   ?allworldcups wdt:P3450 wd:Q19317 . #I filter all the world championships in the variable world Championship
   ?allworldcups wdt:P1346 ?winner . #I get the value of the property winner and I put it on the variable ?winner
   
   # get the label 
   ?allworldcups sc:name ?allworldcupsName  . 
   ?winner sc:name ?winnerName . 
}
ORDER BY ?allworldcupsName 
LIMIT 100


"""

print("Results")
x=run_query(queryString)

Results
[('allworldcups', 'http://www.wikidata.org/entity/Q48432'), ('allworldcupsName', '1930 FIFA World Cup'), ('winner', 'http://www.wikidata.org/entity/Q134916'), ('winnerName', 'Uruguay national football team')]
[('allworldcups', 'http://www.wikidata.org/entity/Q1477177'), ('allworldcupsName', '1934 FIFA World Cup'), ('winner', 'http://www.wikidata.org/entity/Q676899'), ('winnerName', 'Italy national association football team')]
[('allworldcups', 'http://www.wikidata.org/entity/Q131591'), ('allworldcupsName', '1938 FIFA World Cup'), ('winner', 'http://www.wikidata.org/entity/Q676899'), ('winnerName', 'Italy national association football team')]
[('allworldcups', 'http://www.wikidata.org/entity/Q132515'), ('allworldcupsName', '1950 FIFA World Cup'), ('winner', 'http://www.wikidata.org/entity/Q134916'), ('winnerName', 'Uruguay national football team')]
[('allworldcups', 'http://www.wikidata.org/entity/Q131075'), ('allworldcupsName', '1954 FIFA World Cup'), ('winner', 'http://www.wik

In [24]:
#With last query I get the teams of the countries but not the country itself, so I am going to explore the properties of one team. 
#This case the properties of Uruguay Team (Q134916). 
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q134916 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3441'), ('name', 'FIFA country code')]
[('p', 'http://www.wikidata.org/prop/direct/P115'), ('name', 'home venue')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('name', 'owned by')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1352'), ('name', 'ranking')]
[('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('name', 'Twitter username')]
[('p', 'http://www.wikidata.org/prop/direct/P2013'), ('name', 'Facebook ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('name', 'competition class')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3417

In [44]:
#With the last query, I get the country property (P17) that I am gonna use to get the countries instead of the teams. 
queryString = """

SELECT DISTINCT ?allworldcups ?worldcupName ?countryWinner
WHERE {

   #bind something
   ?allworldcups wdt:P3450 wd:Q19317 . #I filter all the world championships in the variable world Championship
   ?allworldcups wdt:P1346 ?winner . #I get the value of the property winner and I put it on the variable ?winner
   ?winner wdt:P17 ?country .  #I get the country and put it on the variable ?country
   
   # get the label 
   ?allworldcups sc:name ?worldcupName  . 
   ?country sc:name ?countryWinner . 
}
ORDER BY ?allworldcupsName 
LIMIT 100


"""

print("Results")
x=run_query(queryString)

Results
[('allworldcups', 'http://www.wikidata.org/entity/Q170645'), ('worldcupName', '2018 FIFA World Cup'), ('countryWinner', 'France')]
[('allworldcups', 'http://www.wikidata.org/entity/Q101730'), ('worldcupName', '1998 FIFA World Cup'), ('countryWinner', 'France')]
[('allworldcups', 'http://www.wikidata.org/entity/Q134202'), ('worldcupName', '1966 FIFA World Cup'), ('countryWinner', 'United Kingdom')]
[('allworldcups', 'http://www.wikidata.org/entity/Q132664'), ('worldcupName', '1970 FIFA World Cup'), ('countryWinner', 'Brazil')]
[('allworldcups', 'http://www.wikidata.org/entity/Q47735'), ('worldcupName', '2002 FIFA World Cup'), ('countryWinner', 'Brazil')]
[('allworldcups', 'http://www.wikidata.org/entity/Q101751'), ('worldcupName', '1994 FIFA World Cup'), ('countryWinner', 'Brazil')]
[('allworldcups', 'http://www.wikidata.org/entity/Q132533'), ('worldcupName', '1958 FIFA World Cup'), ('countryWinner', 'Brazil')]
[('allworldcups', 'http://www.wikidata.org/entity/Q160813'), ('world

In [45]:
#Now I only get the countries and count the times that each country won the world cup. 
queryString = """

SELECT DISTINCT ?countryWinner (COUNT(*) AS ?Times)
WHERE {

   #bind something
   ?allworldcups wdt:P3450 wd:Q19317 . #I filter all the world championships in the variable world Championship
   ?allworldcups wdt:P1346 ?winner . #I get the value of the property winner and I put it on the variable ?winner
   ?winner wdt:P17 ?country . #I get the country and put it on the variable ?country 
   
   # get the label 
   #?allworldcups sc:name ?allworldcupsName  . 
   ?country sc:name ?countryWinner .
}
GROUP BY ?countryWinner
ORDER BY ?countryWinner  
LIMIT 100


"""

print("Results")
x=run_query(queryString)

Results
[('countryWinner', 'Argentina'), ('Times', '2')]
[('countryWinner', 'Brazil'), ('Times', '5')]
[('countryWinner', 'France'), ('Times', '2')]
[('countryWinner', 'Germany'), ('Times', '4')]
[('countryWinner', 'Italy'), ('Times', '4')]
[('countryWinner', 'Spain'), ('Times', '1')]
[('countryWinner', 'United Kingdom'), ('Times', '1')]
[('countryWinner', 'Uruguay'), ('Times', '2')]
8


In [46]:
#Now I only get the ones that won at least two times. 
queryString = """

SELECT DISTINCT ?countryWinner (COUNT(*) AS ?Times)
WHERE {

   #bind something
   ?allworldcups wdt:P3450 wd:Q19317 . #I filter all the world championships in the variable world Championship
   ?allworldcups wdt:P1346 ?winner . #I get the value of the property winner and I put it on the variable ?winner
   ?winner wdt:P17 ?country . #I get the country and put it on the variable ?country 
   
   # get the label 
   #?allworldcups sc:name ?allworldcupsName  . 
   ?country sc:name ?countryWinner .
}
GROUP BY ?countryWinner
HAVING (count(*) >=2)
ORDER BY ?countryWinner  
LIMIT 100


"""

print("Results")
x=run_query(queryString)

Results
[('countryWinner', 'Argentina'), ('Times', '2')]
[('countryWinner', 'Brazil'), ('Times', '5')]
[('countryWinner', 'France'), ('Times', '2')]
[('countryWinner', 'Germany'), ('Times', '4')]
[('countryWinner', 'Italy'), ('Times', '4')]
[('countryWinner', 'Uruguay'), ('Times', '2')]
6


The last query provides me the countries, now I am going to use one of the queries on task 6 that gets the players that won the ballon d'or. 

I'm going to put one query inside other query, the outside query it's gonna get the players and the inside one it's gonna give the countries that won two times the world cup. 

Final query for this task

In [10]:
queryString = """

SELECT DISTINCT ?footballplayers ?footballplayersName 
WHERE {
   
   # This is going to get all the players that won the ballon d'or 
   ?footballplayers wdt:P31 wd:Q5 . #I filter all the humans in the variable footballplayers
   ?footballplayers wdt:P106 wd:Q937857 . #now I add the condition to the occupation has to be "association football player"
   ?footballplayers wdt:P166 wd:Q166177 . #with ballon d'or in the property P166 (award recevied)
   
   #Now I take the countries of the players 
   ?footballplayers wdt:P27 ?country . 
   
   
    {
        #This query is the one that we use before to get the countries that won 2 times the world cup. 
        SELECT DISTINCT ?country (COUNT(*) AS ?Times)
        WHERE {

            #bind something
               ?allworldcups wdt:P3450 wd:Q19317 . #I filter all the world championships in the variable world Championship
               ?allworldcups wdt:P1346 ?winner . #I get the value of the property winner and I put it on the variable ?winner
               ?winner wdt:P17 ?country . #I get the country and put it on the variable ?country
                
        }
        GROUP BY ?country
        HAVING (count(*) >=2)
        ORDER BY ?country  
    }
    
    
   # get the label 
   ?footballplayers sc:name ?footballplayersName . 
   
}
ORDER BY ASC (?footballplayersName)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('footballplayers', 'http://www.wikidata.org/entity/Q164546'), ('footballplayersName', 'Alfredo Di Stéfano')]
[('footballplayers', 'http://www.wikidata.org/entity/Q102027'), ('footballplayersName', 'Fabio Cannavaro')]
[('footballplayers', 'http://www.wikidata.org/entity/Q4457'), ('footballplayersName', 'Franz Beckenbauer')]
[('footballplayers', 'http://www.wikidata.org/entity/Q43744'), ('footballplayersName', 'Gerd Müller')]
[('footballplayers', 'http://www.wikidata.org/entity/Q230046'), ('footballplayersName', 'Gianni Rivera')]
[('footballplayers', 'http://www.wikidata.org/entity/Q13494'), ('footballplayersName', 'Jean-Pierre Papin')]
[('footballplayers', 'http://www.wikidata.org/entity/Q531814'), ('footballplayersName', 'Kaká')]
[('footballplayers', 'http://www.wikidata.org/entity/Q152871'), ('footballplayersName', 'Karl-Heinz Rummenigge')]
[('footballplayers', 'http://www.wikidata.org/entity/Q615'), ('footballplayersName', 'Lionel Messi')]
[('footballplayers', 'http://www.w